# Grupo 20: Thiago e Gabriele

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from sklearn import preprocessing
from sklearn import cluster

In [4]:
df = pd.read_csv('trabalho6_dados_20.csv')
df

,temperatura,vacuo,energia
0,23.82,44.89,445.45
1,22.72,69.84,436.70
2,12.11,41.17,475.53
3,30.27,64.05,438.68
4,15.23,37.87,464.02
...,...,...,...
11477,24.26,61.02,442.86
11478,21.67,69.71,440.16
11479,11.43,40.22,477.50
11480,27.60,69.05,436.08


In [5]:
df.describe()

,temperatura,vacuo,energia
count,11482.000000,11482.000000,11482.000000
mean,19.740275,54.433817,454.117745
std,7.437203,12.682335,17.079184
min,1.810000,25.360000,425.120000
25%,13.670000,41.780000,439.520000
50%,20.550000,52.720000,451.015000
75%,25.760000,66.540000,467.957500
max,37.110000,81.560000,495.760000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11482 entries, 0 to 11481
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperatura  11482 non-null  float64
 1   vacuo        11482 non-null  float64
 2   energia      11482 non-null  float64
dtypes: float64(3)
memory usage: 269.2 KB
